
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: árboles de decisión</h3></font>
***

El objetivo de este *notebook* es que desarrollemos el código para solucionar un problema de clasificación.

## Selección de los datos

In [1]:
## Escogemos los datos
from sklearn.datasets import load_breast_cancer

## Librerías
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report

## Fijamos semilla para reproducir resultados
random.seed(1)

## Control de warnings
import warnings
warnings.filterwarnings('once')

In [ ]:
## Es una función
load_breast_cancer

In [ ]:
## Ayuda de una función
help(load_breast_cancer)

In [4]:
## Ejecutamos la función
data = load_breast_cancer()

In [ ]:
## Vemos qué es: diccionario con diferentes clave-valor
data

In [ ]:
## Miramos qué claves contiene
data.keys()

In [ ]:
## Imprimimos la descripción
print(data["DESCR"])

In [8]:
data = load_breast_cancer(as_frame=True)

In [ ]:
data["data"]

In [ ]:
## Para visualizarlo bien
pd.options.display.max_columns = None
data["data"]

In [ ]:
data["target_names"]

## 0: maligno y 1: benigno

In [ ]:
## Nos falta el target
df = data["data"]
df["class"] = data["target"]

df.head(5)

## Análisis exploratorio

In [ ]:
## Obtención de análisis descriptivo
df.describe().T[["count", "min", "max", "mean", "std"]]

In [ ]:
## Verificamos que no hay valores ausentes
df.isna().sum().sum()

In [ ]:
df.hist(figsize=(10, 20), layout=(-1, 4));
## Poniendo ; evitamos que la última línea salga por pantalla

In [ ]:
## Correlaciones entre las variables
df.corr()

In [ ]:
## Más visual
plt.figure(figsize=(20, 20)) ## Para mostrar todas las variables
sns.heatmap(df.corr(), 
            #cmap = "bwr", 
            vmin = -1, 
            vmax = 1, 
            annot = True, 
            cbar = True);

## Importante poner máximo y mínimo porque sino considera como máximo la máxima correlación que obtenemos
## (que será 1 al considerar correlaciones de una variable con ella misma) y como mínimo la mínima 
## correlación que obtenemos y puede desvirtuar la escala

In [ ]:
## Mismo dibujo pero segmentado por clusters, se reordena creando grupos de variables similares en correlación
plt.figure(figsize=(20, 20)) ## Para mostrar todas las variables
sns.clustermap(df.corr(), cmap = "bwr", vmin = -1, vmax = 1);

## División de los datos

In [18]:
## Importamos las librerías necesarias
from ... import ...

In [ ]:
## Establecemos nombre de target y atributos
target_column = ...
attribute_column = ...

print(f"Target: {target_column}")
print(f"Atributos: {attribute_column}")

In [37]:
## ------------------------------------------- ##
## División en entrenamiento y test
## ------------------------------------------- ##

df_train, df_test = ...

In [38]:
## Separamos target y variables predictoras
y_train = df_train[target_column]
X_train = df_train[attribute_column]

## Otra forma
#X_train = df_train.drop("class", axis=1)

y_test = ...
X_test = ...

In [ ]:
## Comprobamos dimensiones
print(f"Dimensiones train: {...}.")
print(f"Dimensiones test: {...}.")

In [ ]:
## ¿Mantenemos proporciones?
print(f"Proporción del target en train: {y_train.value_counts() / len(y_train)}")
print(f"Proporción del target en test: {...}")

## Entrenamiento de un árbol de clasificación

In [23]:
## Importamos librería para entrenar árbol de clasificación
from ... import ...

In [ ]:
## ------------------------------------------- ##
## Árbol de decisión
## ------------------------------------------- ##

## Instanciamos sin límite de poda
model = ...

## Entrenamos con el conjunto de entrenamiento
...

## Accuracy en train y test
print(f"Accuracy (train, test): {..., ...}")

In [ ]:
## ------------------------------------------- ##
## Visualización del árbol
## ------------------------------------------- ##

## Importamos librería para visualizar el árbol
from ... import ...

plt.figure(figsize=(25,15), dpi=200)
plot_tree(model, 
          rounded=True, 
          proportion=True, 
          label='none',
          feature_names=attribute_column,
          class_names=data["target_names"].tolist(), ## Para poder visualizar con nombres
          filled=True);

## Respuesta a la pregunta: si es que no, hacia la derecha
## Respuesta a la pregunta: si es que sí, hacia la izquierda

In [ ]:
## ------------------------------------------- ##
## Reglas en texto
## ------------------------------------------- ##

## Importamos librería para visualizar las reglas
from ... import ...

## Reglas
texto_modelo = export_text(
                    decision_tree = model,
                    feature_names = attribute_column
               )
print(texto_modelo)

In [ ]:
## ------------------------------------------- ##
## Importancia de los predictores
## ------------------------------------------- ##

importancia_predictores = pd.DataFrame(
                            {'predictor': attribute_column,
                             'importancia': model.feature_importances_}
                            )

print("Importancia de los predictores en el modelo")
print("")

importancia_predictores.sort_values('importancia', ascending=False)

## Optimizamos hiperparámetros

In [ ]:
## ------------------------------------------- ##
## Optimización con grid search
## ------------------------------------------- ##

## Importamos la librería para realizar un grid search
...

## Hacemos el grid search
...

In [ ]:
## Mejor modelo
...

In [30]:
## Lo guardamos para usarlo
model_gridsearch = ...

In [ ]:
## ------------------------------------------- ##
## Métricas
## ------------------------------------------- ##

...

In [ ]:
## ------------------------------------------- ##
## Visualización del árbol
## ------------------------------------------- ##

plt.figure(figsize=(25,15), dpi=200)
plot_tree(model_gridsearch, 
          rounded=True, 
          proportion=True, 
          label='none',
          feature_names=attribute_column,
          class_names=data["target_names"].tolist(),
          filled=True);

## Respuesta a la pregunta: si es que no, hacia la derecha
## Respuesta a la pregunta: si es que sí, hacia la izquierda

In [ ]:
## ------------------------------------------- ##
## Curva ROC
## ------------------------------------------- ##

y_test_proba_predict  = model_gridsearch.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test,
                                 y_test_proba_predict[:,1],
                                 pos_label=1)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, "-o", color='darkorange',
         lw=2, label='model (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = 'random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC at test')
plt.legend(loc="lower right");

## Comparativa con modelo lineal

Para este problema probamos con una regresión logística.

Recordar que en una regresión logística los datos se tienen que normalizar. En este caso, vamos a estandarizar las varibles, es decir, centrarlas en 0 y con desviación estándar 1 para evitar tener rangos de variables muy distintos:

$XtrainEstandarizado = \frac{Xtrain - media(Xtrain)}{std(Xtrain)}$

Destacar que la normalización o estandarización de los datos debe hacerse en el conjunto de entrenamiento, ya que son los datos con los que contamos y sí sabemos su *target* real. Después, esta transformación resultante se aplica sobre el conjunto de test.

In [47]:
## ------------------------------------------- ##
## Estandarización
## ------------------------------------------- ##

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

## Calculo las medias y std de todas las variables en el conjunto de entrenamiento
scaler.fit(X_train)

## Se aplican los valores calculados
X_train_estand = scaler.transform(X_train) ## X_train_estand es la versión estandarizada de X_train
X_test_estand = scaler.transform(X_test) ## X_test_estan es la versión estandarizada de X_test

In [ ]:
X_train.std()

In [ ]:
X_train.mean()

In [ ]:
X_train_estand.mean(axis=0)

In [ ]:
X_train_estand.std(axis=0)

In [ ]:
## ------------------------------------------- ##
## Regresión logística
## ------------------------------------------- ##

from sklearn.linear_model import LogisticRegression

## Instancio la regresión logística
log_model = LogisticRegression()

## Entrenamos con los datos estandarizados
log_model.fit(X_train_estand, y_train)


In [ ]:
## ------------------------------------------- ##
## Calidad predictiva del modelo
## ------------------------------------------- ##

print("Accuracy training = {}".format(log_model.score(X_train_estand, y_train)))
print("Accuracy test = {}".format(log_model.score(X_test_estand, y_test)))

## Calculamos estadístico AUC en train y test
print(f"\nAUC en train: {roc_auc_score(y_train, log_model.predict_proba(X_train_estand)[:, 1])}")
print(f"AUC en test: {roc_auc_score(y_test, log_model.predict_proba(X_test_estand)[:, 1])}")

## Calculamos también precisión y recall en test
print(f"\nPrecisión en test: {precision_score(y_test, log_model.predict(X_test_estand))}")
print(f"Recall en test: {recall_score(y_test, log_model.predict(X_test_estand))}")

## Especificidad (sobre clase 0 que es la maligna)
tn, fp, fn, tp = confusion_matrix(y_test, model_gridsearch.predict(X_test)).ravel() ## Con ravel (de numpy) podemos extender una matriz y poder asignar los valores del numpy 1D
specificity = tn / (tn+fp)
print(f"Especificidad en test: {specificity}\n")

# Represenetación de la matriz de confusión
confusion_matrix_test = confusion_matrix(y_test, log_model.predict(X_test_estand))
print(confusion_matrix_test) # row: real class; column: predicted class

plt.figure(figsize=(4,4))
sns.heatmap(confusion_matrix_test, cmap=plt.cm.rainbow)
plt.title("Matriz de confusión en test")
plt.ylabel('Clase real')
plt.xlabel('Clase predicha');

## SVM

Aplicamos ahora también un SVM para clasificar.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
## ------------------------------------------- ##
## SVM: grid search
## ------------------------------------------- ##

grid = [{'C': [1, 10, 100], 
         'gamma': [0.1, 0.01, 0.001], 
         'kernel': ['rbf', 'poly'],
         'degree': [2, 3],
         'coef0': [1.0]
        },
       ]

clf_gridsearch = GridSearchCV(estimator=SVC(probability=True), param_grid=grid, cv=5)
clf_gridsearch.fit(...) 

## Recordar que si quiero aplicar predict_proba tengo que activar el parámetro probability!

In [ ]:
## Nos quedamos con el mejor
print("Best score = ", clf_gridsearch.best_score_)
clf = clf_gridsearch.best_estimator_
print("Best model: ")
print(clf)

In [ ]:
## ------------------------------------------- ##
## Calidad predictiva del modelo
## ------------------------------------------- ##

print("Accuracy training = {}".format(clf.score(X_train_estand, y_train)))
print("Accuracy test = {}".format(clf.score(X_test_estand, y_test)))

## Calculamos estadístico AUC en train y test
print(f"\nAUC en train: {roc_auc_score(y_train, clf.predict_proba(X_train_estand)[:, 1])}")
print(f"AUC en test: {roc_auc_score(y_test, clf.predict_proba(X_test_estand)[:, 1])}")

## Calculamos también precisión y recall en test
print(f"\nPrecisión en test: {precision_score(y_test, clf.predict(X_test_estand))}")
print(f"Recall en test: {recall_score(y_test, clf.predict(X_test_estand))}")

## Especificidad (sobre clase 0 que es la maligna)
tn, fp, fn, tp = confusion_matrix(y_test, model_gridsearch.predict(X_test)).ravel() ## Con ravel (de numpy) podemos extender una matriz y poder asignar los valores del numpy 1D
specificity = tn / (tn+fp)
print(f"Especificidad en test: {specificity}\n")

# Represenetación de la matriz de confusión
confusion_matrix_test = confusion_matrix(y_test, clf.predict(X_test_estand))
print(confusion_matrix_test) # row: real class; column: predicted class

plt.figure(figsize=(4,4))
sns.heatmap(confusion_matrix_test, cmap=plt.cm.rainbow)
plt.title("Matriz de confusión en test")
plt.ylabel('Clase real')
plt.xlabel('Clase predicha');